In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
google = requests.get("https://developers.google.com")
print("Google:", google.status_code)

Google: 200


In [4]:
NBA = response = requests.get("https://api.sportsdata.io/api/nba/fantasy/json/CurrentSeason")
print("NBA:", NBA.status_code)     # code 401 - you are not authenticated

NBA: 401


In [6]:
rotten_tomato = requests.get("http://api.rottentomatoes.com/api/public/v1.0/lists/movies/box_office.json")
print("Rotten Tomatoes:", rotten_tomato.status_code)      # code 403 - the resources you are trying to access are forbidden

Rotten Tomatoes: 403


In [24]:
# Skyscanner API

url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/SFO-sky/JFK-sky/2021-01-12"
params = {"inboundpartialdate":"2021-01-12"}
headers = {
    "x-rapidapi-host": "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    "x-rapidapi-key": "c9495142cdmshc76277d40553133p1c27acjsnb72f9dd7a59c"}

response = requests.get(url, headers=headers, params=params)

response.json()

{'Quotes': [{'QuoteId': 1,
   'MinPrice': 95,
   'Direct': True,
   'OutboundLeg': {'CarrierIds': [988],
    'OriginId': 81727,
    'DestinationId': 60987,
    'DepartureDate': '2021-01-12T00:00:00'},
   'QuoteDateTime': '2020-12-02T09:29:00'}],
 'Carriers': [{'CarrierId': 988, 'Name': 'Delta'}],
 'Places': [{'Name': 'New York John F. Kennedy',
   'Type': 'Station',
   'PlaceId': 60987,
   'IataCode': 'JFK',
   'SkyscannerCode': 'JFK',
   'CityName': 'New York',
   'CityId': 'NYCA',
   'CountryName': 'United States'},
  {'Name': 'San Francisco International',
   'Type': 'Station',
   'PlaceId': 81727,
   'IataCode': 'SFO',
   'SkyscannerCode': 'SFO',
   'CityName': 'San Francisco',
   'CityId': 'SFOA',
   'CountryName': 'United States'}],
 'Currencies': [{'Code': 'USD',
   'Symbol': '$',
   'ThousandsSeparator': ',',
   'DecimalSeparator': '.',
   'SymbolOnLeft': True,
   'SpaceBetweenAmountAndSymbol': False,
   'RoundingCoefficient': 0,
   'DecimalDigits': 2}]}

In [25]:
import pandas as pd
import json

In [26]:
pd.json_normalize(response.json())

,Quotes,Carriers,Places,Currencies
0,"[{'QuoteId': 1, 'MinPrice': 95, 'Direct': True...","[{'CarrierId': 988, 'Name': 'Delta'}]","[{'Name': 'New York John F. Kennedy', 'Type': ...","[{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa..."


In [27]:
quotes = pd.DataFrame(pd.json_normalize(response.json())["Quotes"][0])
quotes

,QuoteId,MinPrice,Direct,OutboundLeg,QuoteDateTime
0,1,95,True,"{'CarrierIds': [988], 'OriginId': 81727, 'Dest...",2020-12-02T09:29:00


In [31]:
flights = pd.DataFrame(pd.DataFrame(pd.json_normalize(response.json())["Quotes"][0])["OutboundLeg"][0])
flights

,CarrierIds,OriginId,DestinationId,DepartureDate
0,988,81727,60987,2021-01-12T00:00:00


In [29]:
def lowest_price(origin, destination, start, end):
    flights = flight_days(origin, destination, start, end)
    prices = {date:min([quote["MinPrice"] for quote in flights[date]["Quotes"]]) for date in flights for quote in flights[date]["Quotes"]}
    return min(prices, key = prices.get)